In [2]:
import numpy as np
import pandas as pd
import random
import math
import statistics as stat
import matplotlib.pyplot as plt
import itertools
import warnings

In [5]:
def mmn_queueing_jsq_weibull(n,arrival_rate,scale,shape,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the method of JSQ 
    (namely join the shortest queue method), the user needs to provide
    the number of servers (n),arrival rate (lambda) and scale/shape,
    in case of n >= 2, if scale/shape is provided as a scaler, then all servers
    share the same scale/shape, if scale/shape is provided as a vector, then all servers may have different scale/shape,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    There are 6 inputs in this function,
    n:                the number of servers, a positive finite integer
    arrival_rate:     exponential rate of arrival
    scale:            weibull distribution scale
    shape:            weibull distribution shape
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''

    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    lamb =  arrival_rate
    
    if isinstance(scale, list):
        if len(scale) == n:
            scale = scale
        elif (n > 1 and len(scale) == 1):
            scale = n*scale
        else:
            raise Exception("Unmatched cardinality for scale and n")
    elif isinstance(scale, int):
        if n == 1:
            scale = [scale]
        elif n > 1:
            scale = n*[scale]
    else:
        raise Exception("Please use int or list type for scale")
        
    if isinstance(shape, list):
        if len(shape) == n:
            shape = shape
        elif (n > 1 and len(shape) == 1):
            shape = n*shape
        else:
            raise Exception("Unmatched cardinality for shape and n")
    elif isinstance(shape, int):
        if n == 1:
            shape = [shape]
        elif n > 1:
            shape = n*[shape]
    else:
        raise Exception("Please use int or list type for shape")
    
    # Record the start/queue/finish timing for each arrival
    for i in range(1,n+1):
        exec(f'serverlog_{i} = []')
    timing_table = np.zeros(5)
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,3+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1)  # keep track of the number customers
    no_completed_service = np.zeros(n+1)   # keep track of the number of completed services
    arrival_label = 0    # keep track on each arrival's label
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            server_assignment = random.choice(np.where(event_calendar[-1,total_col] == \
                                   min(event_calendar[-1,total_col]))[0])
            no_customers[server_assignment] += 1
            arrival_label += 1
            
            eval(f'serverlog_{server_assignment+1}').append(arrival_label)
            timing_table = np.vstack([timing_table,np.zeros(5)])
            timing_table[-1,0] = arrival_label
            timing_table[-1,1] = event_calendar[-1,0]
            timing_table[-1,4] = server_assignment+1
            
            # If the server is not occupied
            if event_calendar[-1,table_col[server_assignment]] == 0:
                event_calendar[-1,table_col[server_assignment]] += 1
                event_calendar[-1,avail_col[server_assignment]] = event_calendar[-1,0] + \
                                                        scale[server_assignment]*np.random.weibull(shape[server_assignment])
                event_calendar[-1,total_col[server_assignment]] += 1
                
                timing_table[-1,2] = event_calendar[-1,0]
            
            # Else if the server is occupied
            elif event_calendar[-1,table_col[server_assignment]] == 1:
                event_calendar[-1,queue_col[server_assignment]] += 1
                event_calendar[-1,total_col[server_assignment]] += 1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            job_completion = eval(f'serverlog_{server_completion+1}')[0]
            no_completed_service[server_completion] += 1
            
            del eval(f'serverlog_{server_completion+1}')[0]
            timing_table[job_completion,3] = event_calendar[-1,0]
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        scale[server_completion]*np.random.weibull(shape[server_completion])
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
                
                timing_table[eval(f'serverlog_{server_completion+1}')[0],2] = event_calendar[-1,0]
    
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)]))
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    timing_table = pd.DataFrame(timing_table)
    timing_table.columns = ['Arrival_label','Start','Queue','Finish','Server']
    timing_table = timing_table[timing_table['Finish'] != 0]
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time
    awt_data = np.array([])
    for i in range(1,n+1):
        awt_table = timing_table[timing_table['Server']==i]
        if awt_table.shape[0] != 0:
            awt_data = np.append(awt_data,stat.mean(awt_table['Queue']-awt_table['Start']))
        else:
            awt_data = np.append(awt_data,0)
    awt_data = np.append(awt_data,stat.mean(timing_table['Queue']-timing_table['Start']))
    awt = pd.DataFrame(awt_data.round(3)).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time
    act_data = np.array([])
    for i in range(1,n+1):
        act_table = timing_table[timing_table['Server']==i]
        if act_table.shape[0] != 0:
            act_data = np.append(act_data,stat.mean(act_table['Finish']-act_table['Start']))
        else:
            act_data = np.append(act_data,0)
    act_data = np.append(act_data,stat.mean(timing_table['Finish']-timing_table['Start']))
    act = pd.DataFrame(act_data.round(3)).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]

    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "scale": scale,
        "shape": shape,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat


In [6]:
mmn_queueing_jsq_weibull(5,20,3,2,simulation_time=60,simuseed=8)[0]

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,...,Server4,Server4 Available Time,Server4 Queue,Server4 Total,Server5,Server5 Available Time,Server5 Queue,Server5 Total,Grand Queue,Grand Total
1,0.000000,0.012855,60.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
2,0.012855,0.176753,60.0,0.0,NaN,0.0,0.0,1.0,5.379125,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0
3,0.176753,0.187482,60.0,1.0,3.117845,0.0,1.0,1.0,5.379125,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,2.0
4,0.187482,0.191933,60.0,1.0,3.117845,0.0,1.0,1.0,5.379125,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,3.0
5,0.191933,0.275304,60.0,1.0,3.117845,0.0,1.0,1.0,5.379125,0.0,...,1.0,4.126507,0.0,1.0,0.0,NaN,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,59.828844,59.839366,60.0,1.0,60.556717,221.0,222.0,1.0,60.959810,221.0,...,1.0,62.392754,220.0,221.0,1.0,61.005997,221.0,222.0,1104.0,1109.0
1327,59.839366,59.957320,60.0,1.0,60.556717,221.0,222.0,1.0,60.959810,221.0,...,1.0,62.392754,221.0,222.0,1.0,61.005997,221.0,222.0,1105.0,1110.0
1328,59.957320,59.963045,60.0,1.0,60.556717,222.0,223.0,1.0,60.959810,221.0,...,1.0,62.392754,221.0,222.0,1.0,61.005997,221.0,222.0,1106.0,1111.0
1329,59.963045,60.054918,60.0,1.0,60.556717,222.0,223.0,1.0,60.959810,221.0,...,1.0,62.392754,221.0,222.0,1.0,61.005997,221.0,222.0,1107.0,1112.0


In [9]:
mmn_queueing_jsq_weibull(5,20,3,2,simulation_time=60,simuseed=8)[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization
0,0.853,0.719,0.661,0.809


In [10]:
mmn_queueing_jsq_weibull(5,20,3,2,simulation_time=60,simuseed=8)[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Grand Average number in the system
0,1.569,1.157,1.075,1.385,5.187


In [11]:
mmn_queueing_jsq_weibull(5,20,3,2,simulation_time=60,simuseed=8)[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Grand Average queue length
0,0.716,0.439,0.414,0.576,2.145


In [12]:
mmn_queueing_jsq_weibull(5,20,3,2,simulation_time=60,simuseed=8)[4] # 

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Grand Average waiting time
0,0.244,0.076,0.064,0.138,0.111


In [13]:
mmn_queueing_jsq_weibull(5,20,3,2,simulation_time=60,simuseed=8)[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Grand Average cycle time
0,0.538,0.2,0.167,0.332,0.268


In [7]:
mmn_queueing_jsq_weibull(5,20,3,2,simulation_time=60,simuseed=8)[6]

,Server1,Server2,Server3,Server4,Server5,Grand total
Arrived customers,243.0,247.0,239.0,243.0,245.0,1217.0
Completed services,24.0,28.0,20.0,24.0,25.0,121.0
